In [1]:
# This file contains a tests to understand basic inverse dynamics
import numpy as np
import pinocchio as pin
from matplotlib import pyplot as plt

from robot_properties_solo.config import Solo12Config

In [2]:
robot = Solo12Config.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data

eff_arr = ["FL_FOOT", "FR_FOOT", "HL_FOOT", "HR_FOOT"]

f = np.load("dat_file/ik.npz")
q, v = f["xs"][:,:robot.model.nq], f["xs"][:,robot.model.nq:]
a = f["us"]
fff = np.load("dat_file/mom.npz")["F_opt"]
fff = np.reshape(fff, (int(len(fff)/12), 12))

KeyError: 'us is not a file in the archive'

In [ ]:
plt.plot(a[:,2])
# plt.plot(fff[:,2])

In [3]:
for i in range(len(fff)):
    tau = np.zeros(robot.model.nv)
    for j in range(len(eff_arr)):
        J = pin.computeFrameJacobian(rmodel, rdata, q[i], rmodel.getFrameId(eff_arr[j]), pin.LOCAL_WORLD_ALIGNED)
        tau += np.matmul(J.T, np.hstack((fff[i][j*3:(j+1)*3], np.zeros(3))))

    # torque from inverse dynamics
    tau_id = pin.rnea(rmodel, rdata, q[i], v[i], a[i])
    
    return tau_id - tau
    

SyntaxError: 'return' outside function (<ipython-input-3-bede623ef477>, line 10)

In [18]:
q = (Solo12Config.initial_configuration)
J = pin.computeFrameJacobian(rmodel, rdata, np.transpose(q), rmodel.getFrameId("FL_FOOT"), pin.LOCAL_WORLD_ALIGNED)
print(J[0:6,0:6].T)

[[ 1.          0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.        ]
 [ 0.          0.22294615  0.14695     1.          0.          0.        ]
 [-0.22294615  0.         -0.1946      0.          1.          0.        ]
 [-0.14695     0.1946      0.          0.          0.          1.        ]]


In [22]:
J.T[0:6]

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.22294615,  0.14695   ,  1.        ,  0.        ,
         0.        ],
       [-0.22294615,  0.        , -0.1946    ,  0.        ,  1.        ,
         0.        ],
       [-0.14695   ,  0.1946    ,  0.        ,  0.        ,  0.        ,
         1.        ]])